# Supersymmetric Multiline Queues

In [1]:
import itertools 
import copy
from sage.combinat.q_analogues import *
from sage.combinat.sf.sf import *
from sage.combinat.ncsf_qsym.qsym import *
from sage.rings.rational_field import *
from sage.combinat.subset import *
from sage.combinat.permutation import *
from sage.misc.latex import *

import sage.combinat.permutation as permutation
from sage.combinat.sf.macdonald import qt_kostka
from sage.combinat.sf.ns_macdonald import E
from sage.rings.polynomial.polydict import ETuple

coeffs_ring = ZZ['q','t']

q = coeffs_ring.gens()[0]
t = coeffs_ring.gens()[1]

coeffs_field = coeffs_ring.fraction_field()
R=PolynomialRing(coeffs_field,'x1,x2,x3,x4,x5,x6,x7,x8,x9,x10')
xs=list(R.gens())

sym = SymmetricFunctions(coeffs_field)
h = sym.homogeneous()
m = sym.monomial()
s = sym.schur()
e = sym.elementary()
p = sym.power()
Ht = sym.macdonald().Ht()
ModH = sym.macdonald(t=0).Ht()
H = sym.macdonald().H()
Hq = sym.macdonald(q=0,t=q).H()
W = sym.macdonald(t=0).P()
S = sym.macdonald(q=0).S()

MJ = sym.macdonald().J()
HLP = sym.hall_littlewood(t).P()
HLQ = sym.hall_littlewood(t).Q();
HLQp = sym.hall_littlewood(t).Qp();
JJ = sym.jack().J()
qsym = QuasiSymmetricFunctions(coeffs_field)
F = qsym.Fundamental()
M = qsym.Monomial()
QS = qsym.QS()
YQS = qsym.YQS()

%display latex

## Object Construction

In [33]:
# COORDINATES START AT 1
# Coordinates are given in the usual way (i,j) means i to the right, j up

def balls_coordinates(balls):
    coords = []
    w = len(balls)
    l = len(balls[0])
    for i in range(w):
        for j in reversed(range(l)):
            if balls[i][j] != 0: 
                for k in range(balls[i][j]):
                    coords.append([i+1,j+1])
    return(coords)

def balls_coordinates_right_order(balls):
    coords = []
    w = len(balls)
    l = len(balls[0])
    for i in range(w):
        for j in (range(l)):
            if balls[i][j] != 0: 
                for k in range(balls[i][j]):
                    coords.append([i+1,j+1])
    return(coords)
    
def balls_is_valid(balls):
    return(True)

# find the queue index in which elem is
def find_queue(self,elem):
    ind = -1
    for i in range(len(self.queues)):
        queue = self.queues[i]
        if elem in queue:
            ind = i
            break
    if ind == -1:
        return("element not queued")
    else:
        return(ind)  
    
# word is a ternary word with 0,1,2 and 1=(, 2=) and 0=blank, to perform parenthesis matching algorithm
# returns the leftover letters from row1 after matching all possible 1s and 2s, and deleting unmatched 2s
def match_parenthesis(w):
    word = w.copy()
    boo = True
    while boo:
        p1 = -1
        p2 = -1
        for i in range(len(word)):
            if word[i] == 1:
                p1 = i
            elif word[i] == 2:
                p2 = i
            if p1 != -1:
                if p2 != -1 and p2>p1:
                    word[p1] = 0
                    word[p2] = 0        
                    break
                else:
                    l = [j for j in range(p1,len(word)) if word[j] == 2]
                    if l==[]:
                        word[p1] = 0
                    
        if p1==-1:
            boo = False
    return(word)


# word is a ternary word with 0,1,2 and 1=(, 2=) and 0=blank, to perform the epsilon-parenthesis matching algorithm
# returns the list of positions of letters of the word that are paired in the process
def parenthesis_matching(w):
    word = w.copy()
    boo = True
    pairs = []
    while boo:
        p1 = -1
        p2 = -1
        for i in range(len(word)):
            if word[i] == 1:
                p1 = i
            elif word[i] == 2:
                p2 = i
            if p1 != -1:
                if p2 != -1 and p2>p1:
                    pairs.append([p1,p2])
                    word[p1] = 0
                    word[p2] = 0        
                    break
                else:
                    l = [j for j in range(p1,len(word)) if word[j] == 2]
                    if l==[]:
                        p = min([j for j in range(p1) if word[j] == 2])
                        pairs.append([p1,p])
                        word[p1] = 0
                        word[p] = 0        
                        break
                    
        if p1==-1:
            boo = False
    return(pairs)

    
# collapse row 1 on top of row 2
# this method updates row1 and row2
# wit is a boolean variable that says if a collapse occurred or not
def two_row_collapse(row1,row2):
    par_word = []
    wit = False
    # Create the parenthesis word to do the matching algorithm
    for i in range(len(row1)):
        if row1[i]!=0:
            for k in range(row1[i]):
                par_word.append(2)
        else:
            par_word.append(0)
        if row2[i]!=0:
            for k in range(row2[i]):
                par_word.append(1)
        else:
            par_word.append(0)
    # Find indices of each row balls
    row1_indices = []
    row2_indices = []
    
    for j in range(len(par_word)):
        if par_word[j] == 1:
            row2_indices.append(j)
        elif par_word[j] == 2:
            row1_indices.append(j)
    # Apply parenthesis matching
    red_par_word = match_parenthesis(par_word)
    # Find indices on row1 to collapse
    
    coll_indices = []
    
    for j in range(len(red_par_word)):
        if red_par_word[j] == 2:
            if j in row1_indices:
                ind = row1_indices.index(j)
                coll_ind = min([k for k in range(len(row1)) if sum(row1[:k+1])>ind])
                coll_indices.append(coll_ind)
    for i in coll_indices:
        wit = True
        row1[i] -= 1
        row2[i] += 1
    return(row1.copy(),row2.copy(),wit)

# Print a list of lists in a nice way
def print_list_of_lists(L):
    #print("--------------------------")
    print('\n'.join(' '.join('%2d' % x for x in l) for l in L))
    #print("--------------------------")

##   MULTILINE DIAGRAMS CLASS   ##

class SupersymmetricMultilineQueue:
    queues = []
    pairings = []
    labelled_balls = []
    case = 0
    r = 0
    set_case = False
     
    
    # balls may be valid (partition content)
    # size = [width,height]
    def __init__(self,balls,eps):
        self.queues = []
        self.pairings = []
        self.labelled_balls = []
        self.balls = balls
        self.epsilon = eps
        self.size = [len(balls[0]),len(balls)]
        self.balls_coordinates = balls_coordinates_right_order(balls)
        self.QM =[ [] for i in range(sum([sum(balls[i]) for i in range(len(balls))])) ]
        
        # Create the ring for the computations
        xxs = ''
        cont_x = 1
        cont_y = 1
        for i in range(self.size[0]):
            if self.epsilon[i] == 1:
                xxs += 'y'+str(cont_x)+','
                cont_x += 1
            else:
                xxs += 'x'+str(cont_y)+','
                cont_y += 1
        xxs = xxs[:-1]
        self.R=PolynomialRing(coeffs_field,xxs)
        self.xs=list(self.R.gens())
        string = ''
        for i in range(len(self.epsilon)):
            string += str(self.xs[i])+' '
        var(string)
        
    # This method creates the queues
    # 
    # case=0:   pairing to the left with (possibly) wrapping conditions 
    #           pairing is done from right to left
    #           queues are given as lists from top to bottom
    #
    # case=1:   pairing to the left with (possible) wrapping conditions
    #           pairing is done based on the parenthesis matching dictated by epsilon
    #           pairings is a list of tuples of positions in which the particles are paired
    #
    # case=2:   minimal distance insertion-pairing to the left with no-wrapping and NO VERTICAL PAIRING ALLOWED
    #           inserts each ball from right to left and updates the recording tableaux
    #
    # case=3:   pairing to the right with (possibly) wrapping conditions WITH NO VERTICAL PAIRING ALLOWED
    #           pairing is done left to right in each row and priority order is respected
    #           queues are given as sequences of coordinates starting from the top
    #
    # case=4:   minimal distance insertion-pairing to the right with no-wrapping and NO VERTICAL PAIRING ALLOWED
    #           inserts each ball from left to right and updates the recording tableaux
    #
    
    def pair(self,case):
        if not self.set_case:
            self.case = case
            self.set_case = True
        self.queues = []
        
        if case==0:       
            to_queue = self.balls_coordinates.copy()
            cont = True
            while cont:
                b = to_queue[-1]
                queue = [b]
                
                to_queue.remove(b)
                done = False
                b0 = b.copy()
                lab = b0[0]
                while not done:
                    j = b0[0]-1
                    
                    #In here we introduce the dependence on epsilon:
                    
                    #bosonic case:
                    if self.epsilon[b0[1]-1]==0:
                        l = [m for m in range(b0[1]) if ([j,m] in to_queue)]
                    #fermionic case:
                    else:
                        l = [m for m in range(b0[1]+1) if ([j,m] in to_queue)]
                    
                    #if b0 is at the bottom
                    if b0[0] == 1:
                        done = True
                        if (len(queue) == 1):
                            self.labelled_balls.append([b0[0],b0[1],1])
                    #otherwise
                    else:
                        #case 1: wrapping
                        if l==[]:
                            #bosonic case:
                            if self.epsilon[b0[1]-1]==0:
                                c = max([m for m in range(b0[1],self.size[0]+1) if ([j,m] in to_queue)])
                            #fermionic case:
                            else:
                                c = max([m for m in range(b0[1]+1,self.size[0]+1) if ([j,m] in to_queue)])
                        #case 2: non-wrapping
                        else:
                            c = max(l)
                        queue.append([j,c])
                        self.labelled_balls.append([j,c,lab])
                        to_queue.remove([j,c])
                        b0 = [j,c]
                self.queues.append(queue)
                if len(to_queue) == 0:
                    cont = False 
        
        if case == 1:
            for r in range(1,self.size[1]):
                r_up = self.balls[r]
                r_down = self.balls[r-1]
                refs = []
                #create parenthesis word: this depends on epsilon
                w = []
                for i in reversed(range(self.size[0])):
                    #bosonic case
                    if self.epsilon[i]==0:
                        for k in range(r_down[i]):
                            w.append(2)
                            refs.append([i+1,r])
                        for l in range(r_up[i]):
                            w.append(1)
                            refs.append([i+1,r+1])
                    #fermionic case
                    else: 
                        for l in range(r_up[i]):
                            w.append(1)
                            refs.append([i+1,r+1])
                        for k in range(r_down[i]):
                            w.append(2)
                            refs.append([i+1,r])
                
                local_pairs = parenthesis_matching(w)
                
                for pair in local_pairs:
                    self.pairings.append([refs[pair[0]],refs[pair[1]]])
                
                
        if case == 2:
            to_queue = self.balls_coordinates.copy()
            cont = True
            while cont:
                b = to_queue[-1]
                queue = [b]
                
                to_queue.remove(b)
                done = False
                b0 = b.copy()
                lab = b0[0]
                while not done:
                    j = b0[0]-1
                    
                    #In here we introduce the dependence on epsilon:
                    
                    #bosonic case:
                    if self.epsilon[b0[1]-1]==0:
                        l = [m for m in range(b0[1]) if ([j,m] in to_queue)]
                    #fermionic case:
                    else:
                        l = [m for m in range(b0[1],self.size[0]+1) if ([j,m] in to_queue)]
                    
                    #if b0 is at the bottom
                    if b0[0] == 1:
                        done = True
                        if (len(queue) == 1):
                            self.labelled_balls.append([b0[0],b0[1],1])
                    #otherwise
                    else:
                        #case 1: wrapping
                        if l==[]:
                            #bosonic case:
                            if self.epsilon[b0[1]-1]==0:
                                c = max([m for m in range(b0[1],self.size[0]+1) if ([j,m] in to_queue)])
                            #fermionic case:
                            else:
                                c = min([m for m in range(b0[1]) if ([j,m] in to_queue)])
                        #case 2: non-wrapping
                        else:
                            #bosonic case:
                            if self.epsilon[b0[1]-1]==0:
                                c = max(l)
                            #fermionic case:
                            else:
                                c = min(l)
                        queue.append([j,c])
                        self.labelled_balls.append([j,c,lab])
                        to_queue.remove([j,c])
                        b0 = [j,c]
                self.queues.append(queue)
                if len(to_queue) == 0:
                    cont = False 
            
        if case == 3:
            rev_balls = []
            for b in self.balls:
                rev_balls.append(list(reversed(b)))
            self.balls_coordinates = balls_coordinates(rev_balls)
            self.pair(0)
            
            self.balls_coordinates = balls_coordinates(self.balls)
            rev_queues = []
            for queue in self.queues:
                rev_q = []
                for p in queue:
                    rev_q.append([p[0],self.size[0]+1-p[1]])
                rev_queues.append(rev_q)
            
            self.queues = rev_queues
        
        if case == 4:
            rev_balls = []
            for b in self.balls:
                rev_balls.append(list(reversed(b)))
            self.balls_coordinates = balls_coordinates(rev_balls)
            self.pair(2)
            
            rb = []
            for b in self.balls:
                rb.append(list(reversed(b)))
            self.balls = rb
            self.balls_coordinates = balls_coordinates(self.balls)
            rev_queues = []
            for queue in self.queues:
                rev_q = []
                for p in queue:
                    rev_q.append([p[0],self.size[0]+1-p[1]])
                rev_queues.append(rev_q)
            
            self.queues = rev_queues
            
                
    # Collapsing procedure for the ball arrangement.
    # WARNING: This changes the original balls so that the new arrangement is non-wrapping
    #          
    
    def collapse(self):
        L = self.size[1]
        for i in range(1,L):
            for j in reversed(range(1,i+1)):
                row1 = balls[j-1].copy()
                row2 = balls[j].copy()
                [balls[j],balls[j-1],wit] = two_row_collapse(row2,row1)
    
                
    # Compute the recording tableau in the collapsing procedure (case 1) and collapse the ball arrangement.
    
    def recording_and_collapse(self):
        collapsed_balls = [(self.size[0])*[0]]
        for b in self.balls_coordinates:
            new_row = (self.size[0])*[0]
            new_row[b[1]-1] = 1
            
            collapsed_balls.append(new_row)
            #print(collapsed_balls)
            
            if self.case in [0,1,2]:
                for j in reversed(range(1,len(collapsed_balls))):
                    row1 = collapsed_balls[j-1].copy()
                    row2 = collapsed_balls[j].copy()
                    [r2,r1,wit] = two_row_collapse(row2,row1)
                    if not wit:
                        self.QM[j].append(b[0])
                        break
                    elif j == 1:
                        self.QM[j-1].append(b[0])
                    [collapsed_balls[j],collapsed_balls[j-1]] = [r2,r1]


                while (self.size[0])*[0] in collapsed_balls:
                    collapsed_balls.remove((self.size[0])*[0])
                    
            elif self.case in [3,4]:
                for j in reversed(range(1,len(collapsed_balls))):
                    row1 = collapsed_balls[j-1].copy()
                    row2 = collapsed_balls[j].copy()
                    [r2,r1,wit] = two_row_collapse(row2,row1)
                    if not wit:
                        self.QM[j].append(b[0])
                        break
                    elif j == 1:
                        self.QM[j-1].append(b[0])
                    [collapsed_balls[j],collapsed_balls[j-1]] = [r2,r1]


                while (self.size[0])*[0] in collapsed_balls:
                    collapsed_balls.remove((self.size[0])*[0])
            
            #print(collapsed_balls)
        
        while [] in self.QM:
            self.QM.remove([])
            
        self.balls = collapsed_balls.copy()
    
    # Draw a queue between pi and pf in a drawing win
    # queues must be ordered from top to bottom
    # case : see pairing cases

    def draw_queue(self,win,pi,pf,case,offset,cells):
        r = self.r
        xi = pi[1]-0.5
        yi = pi[0]-0.5
        xf = pf[1]-0.5
        yf = pf[0]-0.5
        if self.case == 0 or self.case == 1:
            if xi < xf:                
                #bosonic case
                if self.epsilon[pi[1]-1] == 0:
                    win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('blue'))
                    win += line([(xi+offset,yi-0.5+offset),(0,yi-0.5+offset)],color=Color('blue'))
                    win += line([(self.size[0],yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('blue'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('blue'))
                #fermionic case    
                else:
                    win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('red'))
                    win += line([(xi+offset,yi-0.5+offset),(0,yi-0.5+offset)],color=Color('red'))
                    win += line([(self.size[0],yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('red'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('red'))
            
            elif xi == xf:                
                #bosonic case
                if self.epsilon[pi[1]-1] == 0:
                    win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('blue'))
                    win += line([(xi+offset,yi-0.5+offset),(0,yi-0.5+offset)],color=Color('blue'))
                    win += line([(self.size[0],yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('blue'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('blue'))
                #fermionic case    
                else:
                    win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('red'))
                    win += line([(xi+offset,yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('red'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('red'))
               
            else:
                #bosonic case
                if self.epsilon[pi[1]-1] == 0:
                    win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('blue'))
                    win += line([(xi+offset,yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('blue'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('blue'))
                #fermionic case    
                else:
                    win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('red'))
                    win += line([(xi+offset,yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('red'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('red'))
            return(win)
        
        elif self.case == 2:
            if xi < xf:                
                #bosonic case
                if self.epsilon[pi[1]-1] == 0:
                    win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('blue'))
                    win += line([(xi+offset,yi-0.5+offset),(0,yi-0.5+offset)],color=Color('blue'))
                    win += line([(self.size[0],yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('blue'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('blue'))
                #fermionic case    
                else:
                    win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('red'))
                    win += line([(xi+offset,yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('red'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('red'))
                    
            
            elif xi == xf:                
                #bosonic case
                if self.epsilon[pi[1]-1] == 0:
                    win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('blue'))
                    win += line([(xi+offset,yi-0.5+offset),(0,yi-0.5+offset)],color=Color('blue'))
                    win += line([(self.size[0],yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('blue'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('blue'))
                #fermionic case    
                else:
                    win += line([(xi+offset,yi-self.r),(xf+offset,yf+self.r)],color=Color('red'))
                    win += line([(xi+offset,yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('red'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('red'))
               
            else:
                #bosonic case
                if self.epsilon[pi[1]-1] == 0:
                    win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('blue'))
                    win += line([(xi+offset,yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('blue'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('blue'))
                #fermionic case    
                else:
                    win += line([(xi+offset,yi-self.r),(xi+offset,yi-0.5+offset)],color=Color('red'))
                    win += line([(xi+offset,yi-0.5+offset),(self.size[0],yi-0.5+offset)],color=Color('red'))
                    win += line([(0,yi-0.5+offset),(xf+offset,yi-0.5+offset)],color=Color('red'))
                    win += line([(xf+offset,yi-0.5+offset),(xf+offset,yf+self.r)],color=Color('red'))
            return(win)
            
    # 
    def draw_pairings(self,win):
        for r in range(1,self.size[1]):
            pairings_row_r = [pairing for pairing in self.pairings if (pairing[0][1] == r+1)]
            initial_coords = [p[0] for p in pairings_row_r]
            mults_or = [len([pai for pai in pairings_row_r if pai[0] == c]) for c in initial_coords]
            mults_var = [len([pai for pai in pairings_row_r if pai[0] == c]) for c in initial_coords]
            cont = -1
            for i in range(len(pairings_row_r)):  
                M = len(pairings_row_r)
                cont += 1
                p = pairings_row_r[i]
                r = self.r
                pi = p[0]
                pf = p[1]
                
                index_mult = initial_coords.index(pi)
                N = mults_or[index_mult]
                x = mults_var[index_mult]
                if N > 1:
                    offset_x = (0.1)*((2*math.floor(N/2)*(1.0/(N-1)))*(x-1) - math.floor(N/2))
                    offset_y = (-0.1)*((2*math.floor(N/2)*(1.0/(N-1)))*(x-1) - math.floor(N/2))
                elif M > 1:
                    offset_x = 0
                    offset_y = (0.1)*((2*math.floor(M/2)*(1.0/(M-1)))*(cont) - math.floor(M/2))
                else:
                    offset_x = 0
                    offset_y = 0
                mults_var[index_mult] -= 1
                
                xi = pi[0]-0.5
                yi = pi[1]-0.5
                xf = pf[0]-0.5
                yf = pf[1]-0.5
                
                if xi < xf:       
                    #bosonic case
                    if self.epsilon[pi[0]-1] == 0:
                        win += line([(xi+offset_x,yi-self.r),(xi+offset_x,yi-0.5+offset_y)],color=Color('blue'))
                        win += line([(xi+offset_x,yi-0.5+offset_y),(0,yi-0.5+offset_y)],color=Color('blue'))
                        win += line([(self.size[0],yi-0.5+offset_y),(xf+offset_x,yi-0.5+offset_y)],color=Color('blue'))
                        win += line([(xf+offset_x,yi-0.5+offset_y),(xf+offset_x,yf+self.r)],color=Color('blue'))
                    #fermionic case    
                    else:
                        win += line([(xi+offset_x,yi-self.r),(xi+offset_x,yi-0.5+offset_y)],color=Color('red'))
                        win += line([(xi+offset_x,yi-0.5+offset_y),(0,yi-0.5+offset_y)],color=Color('red'))
                        win += line([(self.size[0],yi-0.5+offset_y),(xf+offset_x,yi-0.5+offset_y)],color=Color('red'))
                        win += line([(xf+offset_x,yi-0.5+offset_y),(xf+offset_x,yf+self.r)],color=Color('red'))

                elif xi == xf:   
                    #bosonic case
                    if self.epsilon[pi[0]-1] == 0:
                        win += line([(xi+offset_x,yi-self.r),(xi+offset_x,yi-0.5+offset_y)],color=Color('blue'))
                        win += line([(xi+offset_x,yi-0.5+offset_y),(0,yi-0.5+offset_y)],color=Color('blue'))
                        win += line([(self.size[0],yi-0.5+offset_y),(xf+offset_x,yi-0.5+offset_y)],color=Color('blue'))
                        win += line([(xf+offset_x,yi-0.5+offset_y),(xf+offset_x,yf+self.r)],color=Color('blue'))
                    #fermionic case    
                    else:
                        win += line([(xi+offset_x,yi-self.r),(xi+offset_x,yi-0.5+offset_y)],color=Color('red'))
                        win += line([(xi+offset_x,yi-0.5+offset_y),(xf+offset_x,yi-0.5+offset_y)],color=Color('red'))
                        win += line([(xf+offset_x,yi-0.5+offset_y),(xf+offset_x,yf+self.r)],color=Color('red'))

                else:
                    #bosonic case
                    if self.epsilon[pi[0]-1] == 0:
                        win += line([(xi+offset_x,yi-self.r),(xi+offset_x,yi-0.5+offset_y)],color=Color('blue'))
                        win += line([(xi+offset_x,yi-0.5+offset_y),(xf+offset_x,yi-0.5+offset_y)],color=Color('blue'))
                        win += line([(xf+offset_x,yi-0.5+offset_y),(xf+offset_x,yf+self.r)],color=Color('blue'))
                    #fermionic case    
                    else:
                        win += line([(xi+offset_x,yi-self.r),(xi+offset_x,yi-0.5+offset_y)],color=Color('red'))
                        win += line([(xi+offset_x,yi-0.5+offset_y),(xf+offset_x,yi-0.5+offset_y)],color=Color('red'))
                        win += line([(xf+offset_x,yi-0.5+offset_y),(xf+offset_x,yf+self.r)],color=Color('red'))
        return(win)
                    
    # Gives the drawing of a MLQ that can be paired or not (in which case just shows the ball arrangement)             
    def draw(self,radius,cells):
        win = Graphics()
        win.set_aspect_ratio(1)
        self.r = radius
        w = self.size[1]
        h = self.size[0]
        
        #Draw gray grid        
        for i in range(0,h+1,1):
            win += line([(i,0),(i,w)],color=Color('lightgray'))
        for i in range(0,w+1,1):    
            win += line([(0,i),(h,i)],color=Color('lightgray'))
            
        #Some important parameters
        qs = [i for i in self.queues if len(i) > 1]
        sqs = [i for i in self.queues if len(i) == 1]
        N = len(qs)
        #maximum offset to avoid queues to be intersecting in the drawing
        par = (-1)/(8)
        
        #Draw ball arrangement
        for b in self.balls_coordinates:
            yb = b[0]
            xb = b[1]
            num = self.balls_coordinates.count(b)
            
            if cells == "numbers":
                if self.epsilon[xb-1] == 0:
                    lab = str(num)
                    win+= text(lab,(xb-0.5,yb-0.5),color = Color('blue'),fontsize = 16,rotation = 0)
                else:
                    lab = str(num)
                    win+= circle((xb-0.5,yb-0.5), self.r, fill = True, color = Color('red'))
                    #win+= text(lab,(xb-0.5,yb-0.5),color = Color('red'),fontsize = 20,rotation = 0)

                #To add colors to certain rows
                
                #if (yb < 2):
                #    win+= text(lab,(xb-0.5,yb-0.5),color = Color('blue'),fontsize = 18,rotation = 0, fontweight = 'bold')
                #else:
                #    win+= text(lab,(xb-0.5,yb-0.5),color = Color('red'),fontsize = 18,rotation = 0)
                    
        #Draw queues     
        if self.case == 0:
            for k in range(len(self.queues)):
                    queue = self.queues[k] 
                    l = len(queue)
                    if cells == "balls":
                        if l == 1:
                            num = sqs.count(queue)
                            for k in range(num):
                                off = (par)-(2*par/(1.0*(N-1)))*(k)
                                xb = queue[0][1]
                                yb = queue[0][0]
                                win+= circle((xb+off-0.5,yb-0.5), self.r, fill = True, color = Color('black'))
                    if l > 1:                    
                        for j in range(1,l):                    

                            #offset function
                            off = (par)-(2*par/(1.0*(N-1)))*(k)

                            #draw the line between elements of the queue
                            win = self.draw_queue(win,queue[j-1],queue[j],self.case,off,cells)
    
        elif self.case == 2:
            for k in range(len(self.queues)):
                    queue = self.queues[k] 
                    l = len(queue)
                    if cells == "balls":
                        if l == 1:
                            num = sqs.count(queue)
                            for k in range(num):
                                off = (par)-(2*par/(1.0*(N-1)))*(k)
                                xb = queue[0][1]
                                yb = queue[0][0]
                                win+= circle((xb+off-0.5,yb-0.5), self.r, fill = True, color = Color('black'))
                    if l > 1:                    
                        for j in range(1,l):                    

                            #offset function
                            off = (par)-(2*par/(1.0*(N-1)))*(k)

                            #draw the line between elements of the queue
                            win = self.draw_queue(win,queue[j-1],queue[j],self.case,off,cells)
        
        elif self.case == 1:
            win = self.draw_pairings(win)

        win.axes(False)
        win.show()
        
    def cw(self):
        word= []
        for i in reversed(range(self.size[0])):
            for j in range(self.size[1]):
                for k in range(self.balls[j][i]):
                    word.append(j+1)
        return(word)
            
    def partition(self):
        lam_prime = [sum(b) for b in self.balls]
        lam_p = Partition(lam_prime)
        return(lam_p.conjugate())

    def is_non_wrapping(self):
        bol = True
        for p in self.pairings:
            #bosonic case 
            if self.epsilon[p[0][0]-1] == 0 and p[0][0] <= p[1][0]:
                bol = False
                break
            #fermionic case
            if self.epsilon[p[0][0]-1] == 1 and p[0][0] < p[1][0]:
                bol = False
                break
        return(bol)
    
    def forward_ringing(self,a0):
        balls_new = self.balls.copy()
        
        ring_path = [a0]
        for r in range(1,self.size[1]+1):
            ai = ring_path[r-1]
            #Case 1: current position is fermionic
            if self.epsilon[ai-1] == 1:
                #Case 1a: cell is ocuppied
                if balls[r-1][ai-1] > 0:
                    ring_path.append(ai) 
                #Case 1b: cell is empty
                else:
                    new_col = (ai-1) % self.size[0]
                    if new_col == 0:
                        new_col = self.size[0]
                    ring_path.append(new_col) 
            #Case 2: current position is bosonic
            elif self.epsilon[ai-1] == 0:
                #Case 2a: cell is ocuppied
                if balls[r-1][ai-1] > 0:
                    new_col = (ai+1) % self.size[0]
                    if new_col == 0:
                        new_col = self.size[0]
                    ring_path.append(new_col) 
                #Case 2b: cell is empty
                else:
                    ring_path.append(ai)
                    
        print(ring_path)
        
        for i in range(len(ring_path)-1):
            ci = ring_path[i]
            cf = (ci+1) % self.size[0]
            if cf == 0:
                cf = self.size[0]
            
            #Case 1
            if self.epsilon[cf-1] == 0 and self.balls[i][ci-1] > 0:
                balls_new[i][cf-1] += 1
                balls_new[i][ci-1] -= 1
                
            elif self.epsilon[cf-1] == 1 and self.balls[i][ci-1] > 0 and self.balls[i][cf-1] == 0:
                balls_new[i][cf-1] += 1
                balls_new[i][ci-1] -= 1
        
        if self.epsilon[ring_path[-2]-1] == 1:
            ans = (ring_path[-1]+1) % self.size[0]
            if ans == 0:
                ans = self.size[0]
            return([balls_new,ans])
        else:
            return([balls_new,ring_path[-1]])
    
    def reverse_ringing(self,b0):
        balls_new = self.balls.copy()
        
        ring_path = [b0]
        for r in range(1,self.size[1]+1):
            bi = ring_path[r-1]
            #Case 1: current position is fermionic
            if self.epsilon[bi-1] == 1:
                #Case 1a: cell is ocuppied
                if balls[self.size[1] - r][bi-1] > 0:
                    ring_path.append(bi) 
                #Case 1b: cell is empty
                else:
                    new_col = (bi+1) % self.size[0]
                    if new_col == 0:
                        new_col = self.size[0]
                    ring_path.append(new_col) 
            #Case 2: current position is bosonic
            elif self.epsilon[bi-1] == 0:
                #Case 2a: cell is ocuppied
                if balls[self.size[1] - r][bi-1] > 0:
                    new_col = (bi-1) % self.size[0]
                    if new_col == 0:
                        new_col = self.size[0]
                    ring_path.append(new_col) 
                #Case 2b: cell is empty
                else:
                    ring_path.append(bi)
        
        print(ring_path)
        
        for i in range(len(ring_path)-1):
            ci = ring_path[i]
            cf = (ci-1) % self.size[0]
            if cf == 0:
                cf = self.size[0]
            
            #Case 1
            if self.epsilon[cf-1] == 0 and self.balls[self.size[1] - i -1][ci-1] > 0:
                balls_new[self.size[1] - i -1][cf-1] += 1
                balls_new[self.size[1] - i -1][ci-1] -= 1
            
            #Case 2
            elif self.epsilon[cf-1] == 1 and self.balls[self.size[1] - i -1][ci-1] > 0 and self.balls[self.size[1] - i -1][cf-1] == 0:
                balls_new[self.size[1] - i -1][cf-1] += 1
                balls_new[self.size[1] - i -1][ci-1] -= 1
        
        if self.epsilon[ring_path[-2]-1] == 1:
            ans = (ring_path[-1]-1) % self.size[0]
            if ans == 0:
                ans = self.size[0]
            return([balls_new,ans])
        else:
            return([balls_new,ring_path[-1]])
        
        
    def forward_PUSH_ringing(self,a0):
        balls_new = self.balls.copy()
        
        ring_path = [a0]
        for r in range(1,self.size[1]+1):
            ai = ring_path[r-1]
            #Case 1: current position is fermionic
            if self.epsilon[ai-1] == 1:
                #Case 1a: cell is ocuppied
                if balls[r-1][ai-1] > 0:
                    ops1 = [c for c in range(ai-1) if (self.epsilon[c] == 0) or (self.epsilon[c] == 1 and self.balls[r-1][c] == 0)]
                    ops2 = [c for c in range(ai,self.size[0]) if (self.epsilon[c] == 0) or (self.epsilon[c] == 1 and self.balls[r-1][c] == 0)]
#                     #To the left:
#                     if not len(ops1) == 0:
#                         new_col = (max(ops1)+1) % self.size[0]
#                         if new_col == 0:
#                             new_col = self.size[0]
#                         ring_path.append(new_col) 
#                     else:
#                         new_col = (max(ops2)+1) % self.size[0]
#                         if new_col == 0:
#                             new_col = self.size[0]
#                         ring_path.append(new_col) 

                    #To the right:
                    if not len(ops2) == 0:
                        new_col = (min(ops2)+1) % self.size[0]
                        if new_col == 0:
                            new_col = self.size[0]
                        ring_path.append(new_col) 
                    else:
                        new_col = (min(ops1)+1) % self.size[0]
                        if new_col == 0:
                            new_col = self.size[0]
                        ring_path.append(new_col)
                
                #Case 1b: cell is empty
                else:
                    ring_path.append(ai) 
                    
            #Case 2: current position is bosonic
            elif self.epsilon[ai-1] == 0:
                #Case 2a: cell is ocuppied
                if balls[r-1][ai-1] > 0:
                    ops1 = [c for c in range(ai-1) if (self.epsilon[c] == 0) or (self.epsilon[c] == 1 and self.balls[r-1][c] == 0)]
                    ops2 = [c for c in range(ai,self.size[0]) if (self.epsilon[c] == 0) or (self.epsilon[c] == 1 and self.balls[r-1][c] == 0)]
                    
                    #To the right:
                    if not len(ops2) == 0:
                        new_col = (min(ops2)+1) % self.size[0]
                        if new_col == 0:
                            new_col = self.size[0]
                        ring_path.append(new_col) 
                    else:
                        new_col = (min(ops1)+1) % self.size[0]
                        if new_col == 0:
                            new_col = self.size[0]
                        ring_path.append(new_col)
                    
#                     #To the left:
#                     if not len(ops1) == 0:
#                         new_col = (max(ops1)+1) % self.size[0]
#                         if new_col == 0:
#                             new_col = self.size[0]
#                         ring_path.append(new_col) 
#                     else:
#                         new_col = (max(ops2)+1) % self.size[0]
#                         if new_col == 0:
#                             new_col = self.size[0]
#                         ring_path.append(new_col)
                        
                #Case 2b: cell is empty
                else:
                    ring_path.append(ai)
                    
        print(ring_path)
        
        for i in range(len(ring_path)-1):
            ci = ring_path[i]
            cf = ring_path[i+1]
            #Case 1
            if not ci == cf:
                balls_new[i][cf-1] += 1
                balls_new[i][ci-1] -= 1
        return([balls_new,ring_path[-1]])
    
    def reverse_PUSH_ringing(self,b0):
        balls_new = self.balls.copy()
        
        ring_path = [b0]
        for r in range(1,self.size[1]+1):
            bi = ring_path[r-1]
            #Case 1: current position is fermionic
            if self.epsilon[bi-1] == 1:
                #Case 1a: cell is ocuppied
                if balls[self.size[1] - r][bi-1] > 0:
                    ring_path.append(bi) 
                #Case 1b: cell is empty
                else:
                    new_col = (bi+1) % self.size[0]
                    if new_col == 0:
                        new_col = self.size[0]
                    ring_path.append(new_col) 
            #Case 2: current position is bosonic
            elif self.epsilon[bi-1] == 0:
                #Case 2a: cell is ocuppied
                if balls[self.size[1] - r][bi-1] > 0:
                    new_col = (bi-1) % self.size[0]
                    if new_col == 0:
                        new_col = self.size[0]
                    ring_path.append(new_col) 
                #Case 2b: cell is empty
                else:
                    ring_path.append(bi)
        
        print(ring_path)
        
        for i in range(len(ring_path)-1):
            ci = ring_path[i]
            cf = (ci-1) % self.size[0]
            if cf == 0:
                cf = self.size[0]
            
            #Case 1
            if self.epsilon[cf-1] == 0 and self.balls[self.size[1] - i -1][ci-1] > 0:
                balls_new[self.size[1] - i -1][cf-1] += 1
                balls_new[self.size[1] - i -1][ci-1] -= 1
            
            #Case 2
            elif self.epsilon[cf-1] == 1 and self.balls[self.size[1] - i -1][ci-1] > 0 and self.balls[self.size[1] - i -1][cf-1] == 0:
                balls_new[self.size[1] - i -1][cf-1] += 1
                balls_new[self.size[1] - i -1][ci-1] -= 1
        
        if self.epsilon[ring_path[-2]-1] == 1:
            ans = (ring_path[-1]-1) % self.size[0]
            if ans == 0:
                ans = self.size[0]
            return([balls_new,ans])
        else:
            return([balls_new,ring_path[-1]])
        
    
    def particles(self):
        state = [[] for k in range(self.size[0])]
        for q in self.queues:
            if q[-1][0] == 1:
                state[q[-1][1]-1].append(q[0][0])
        return(state)
            
        
    ##  ALGEBRAIC METHODS  ##
    
    #returns the x-weight of the multiline queue. Does not need to be paired
    def xy_weight(self):
        wei = self.xs[0]**0
        for b in self.balls_coordinates:
            wei *= self.xs[b[1]-1]
        return(wei)
    
    def x_exponent(self):
        exp = [0 for k in range(self.size[0])]
        for b in self.balls_coordinates:
            if self.epsilon[b[1]-1] == 1:
                exp[b[1]-1] += 1
        return([e for e in exp if not e==0])
        #return(exp)
    
    def y_exponent(self):
        exp = [0 for k in range(self.size[0])]
        for b in self.balls_coordinates:
            if self.epsilon[b[1]-1] == 0:
                exp[b[1]-1] += 1
        return([e for e in exp if not e==0])
        #return(exp)
        
    
    #returns the x-weight of the multiline queue. Requires pairing
    # case: see pairing cases
    def qweight(self):
        wei = q**0
        #q statistic calculation depends on the pairing algorithm
        if self.case == 0 or self.case == 1 or self.case == 2:
            for queue in self.queues:
                l = queue[0][0]
                for i in range(1,len(queue)):
                    ci = queue[i-1][1]
                    cf = queue[i][1]
                    if cf >= ci:
                        wei *= q**(l+1-queue[i-1][0])
                        
        elif self.case == 3 or self.case == 4:
            for queue in self.queues:
                l = queue[0][0]
                for i in range(1,len(queue)):
                    ci = queue[i-1][1]
                    cf = queue[i][1]
                    if cf <= ci:
                        wei *= q**(l+1-queue[i-1][0])
        return(wei)
    
    
    def comaj(self):
        wei = q**0
        #q statistic calculation depends on the pairing algorithm
        if self.case == 0:
            for queue in self.queues:
                l = queue[0][0]
                for i in range(1,len(queue)):
                    ci = queue[i-1][1]
                    cf = queue[i][1]
                    # Case 1: initial particle is fermionic and pairing wraps
                    if self.epsilon[ci-1] == 1 and cf > ci:
                        wei *= q**(l+1-queue[i-1][0])
                        
                    # Case 2: initial particle is bosonic and pairing does not wrap
                    elif self.epsilon[ci-1] == 0 and cf >= ci:
                        wei *= q**(l+1-queue[i-1][0])
        return(wei)
        

## Algebraic Combinatorics

In [14]:
##   "Schur" functions   ##

def schur_super_mlq(n,shape,epsilon,disp):
    part = Partition(shape)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        num_list = []
        local_list = []
        for j in range(n):
            for k in range(i):
                num_list.append(j)    
                
        for sub in Subsets(num_list,submultiset=True):
            sub_list = list(sub)
            if len(sub_list) == i:
                is_valid = True
                for col in range(n):
                    if epsilon[col] == 1 and col in sub_list:
                        if sub_list.count(col) > 1:
                            is_valid = False
                            break
                if is_valid:
                    local_list.append(sub_list)  
        aux_list.append(local_list)

    ball_arr_coords = itertools.product(*aux_list)
    
    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for p in row:
                ball_row[p] += 1
            ball_arr.append(ball_row.copy())
        ball_arrangements.append(ball_arr)

    # Create all MLDs with n columns of given shape & compute the polynomial
    poly = 0

    for balls in ball_arrangements:
        
        SuperM = SupersymmetricMultilineQueue(balls.copy(),epsilon)
    
        SuperM.pair(1)
        r=0.2

        if SuperM.is_non_wrapping():
            if disp:
                SuperM.draw(r,"numbers")
                display(SuperM.xy_weight())
            poly += SuperM.xy_weight()

    return(poly)
#     # Convert polynomial to symmetric function
#     f = sym.from_polynomial(poly)
#     return(s(f))


# Pairs

def equality_pairs(n,shape,eps1,eps2):
    part = Partition(shape)
    mlq_shape = list(part.conjugate())

    aux_list1 = []
    ball_arrangements1 = []
    
    aux_list2 = []
    ball_arrangements2 = []

    for i in mlq_shape:
        num_list = []
        local_list1 = []
        local_list2 = []
        for j in range(n):
            for k in range(i):
                num_list.append(j)    
                
        for sub in Subsets(num_list,submultiset=True):
            sub_list = list(sub)
            if len(sub_list) == i:
                is_valid1 = True
                is_valid2 = True
                for col in range(n):
                    if eps1[col] == 1 and col in sub_list:
                        if sub_list.count(col) > 1:
                            is_valid1 = False
                    if eps2[col] == 1 and col in sub_list:
                        if sub_list.count(col) > 1:
                            is_valid2 = False
                if is_valid1:
                    local_list1.append(sub_list) 
                if is_valid2:
                    local_list2.append(sub_list)
                    
        aux_list1.append(local_list1)
        aux_list2.append(local_list2)

    ball_arr_coords1 = itertools.product(*aux_list1)
    ball_arr_coords2 = itertools.product(*aux_list2)
    
    for tup in ball_arr_coords1:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for p in row:
                ball_row[p] += 1
            ball_arr.append(ball_row.copy())
        ball_arrangements1.append(ball_arr)
        
    for tup in ball_arr_coords2:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for p in row:
                ball_row[p] += 1
            ball_arr.append(ball_row.copy())
        ball_arrangements2.append(ball_arr)

    for balls1 in ball_arrangements1:
        r=0.2
        SuperM1 = SupersymmetricMultilineQueue(balls1.copy(),eps1)
        SuperM1.pair(1)
        xw1 = SuperM1.x_exponent()
        yw1 = SuperM1.y_exponent()
        
        if(SuperM1.is_non_wrapping()):
            for balls2 in ball_arrangements2:
                SuperM2 = SupersymmetricMultilineQueue(balls2.copy(),eps2)
                SuperM2.pair(1)
                xw2 = SuperM2.x_exponent()
                yw2 = SuperM2.y_exponent()
                
                if(SuperM2.is_non_wrapping()):
                    if xw1 == xw2 and yw1 == yw2:
                        display(SuperM1.xy_weight())
                        SuperM1.draw(r,"numbers")
                        SuperM2.draw(r,"numbers")
                        
                        
def supers_with_exponents(n,shape,epsilon,x_exp,y_exp):
    part = Partition(shape)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        num_list = []
        local_list = []
        for j in range(n):
            for k in range(i):
                num_list.append(j)    
                
        for sub in Subsets(num_list,submultiset=True):
            sub_list = list(sub)
            if len(sub_list) == i:
                is_valid = True
                for col in range(n):
                    if epsilon[col] == 1 and col in sub_list:
                        if sub_list.count(col) > 1:
                            is_valid = False
                            break
                if is_valid:
                    local_list.append(sub_list)  
        aux_list.append(local_list)

    ball_arr_coords = itertools.product(*aux_list)
    
    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for p in row:
                ball_row[p] += 1
            ball_arr.append(ball_row.copy())
        ball_arrangements.append(ball_arr)
        
    print(len(ball_arrangements))
    cont = 0
    for balls in ball_arrangements:
        SuperM = SupersymmetricMultilineQueue(balls.copy(),epsilon)
        SuperM.pair(1)
        r=0.2
        xw = SuperM.x_exponent()
        yw = SuperM.y_exponent()
        
        if SuperM.is_non_wrapping():
            if (xw == x_exp and yw == y_exp):
                SuperM.draw(r,"numbers")
                cont +=1
    print("Total Found = "+str(cont))
        
        
def contained_in(lam,mu):
    if len(lam) < len(mu):
        return(False)
    else:
        ans = True
        for i in range(len(mu)):
            if lam[i] < mu[i]:
                ans = False
                break
        
        return(ans)
        
# n is the number of x's
# m is the number of y's
# lam is the shape
def hook_Schur_polynomial(lam,n,m):
    
    #CHANGED LABELS FOR CONVENIENCE!!!
    
    x_alp = ["x"+str(j) for j in range(1,n+1)]
    y_alp = ["y"+str(j) for j in range(1,m+1)]

    coeffs_ring = ZZ['q','t']

    q = coeffs_ring.gens()[0]
    t = coeffs_ring.gens()[1]

    coeffs_field = coeffs_ring.fraction_field()
    S=PolynomialRing(coeffs_field,x_alp+y_alp)
    xs=list(S.gens())
    
    sym = SymmetricFunctions(coeffs_field)
    
    N = sum(lam)
    
    poly = 0
    
    for k in range(N+1):
        for part in Partitions(k):
            l_part = list(part)       

            if contained_in(lam,part):
                lam_prime = Partition(lam).conjugate()
                part_prime = part.conjugate()
                sP = SkewPartition([lam_prime,part_prime])

                poly += S(s(part).expand(n,alphabet = xs[0:n]))*S(s(sP).expand(m,alphabet = xs[n:n+m]))
    
    return(poly)
    

## Modified Hall-Littlewood Polynomials ##

def comaj_gen_fun(n,shape,epsilon):
    part = Partition(shape)
    mlq_shape = list(part.conjugate())

    aux_list = []
    ball_arrangements = []

    for i in mlq_shape:
        num_list = []
        local_list = []
        for j in range(n):
            for k in range(i):
                num_list.append(j)    
                
        for sub in Subsets(num_list,submultiset=True):
            sub_list = list(sub)
            if len(sub_list) == i:
                is_valid = True
                for col in range(n):
                    if epsilon[col] == 1 and col in sub_list:
                        if sub_list.count(col) > 1:
                            is_valid = False
                            break
                if is_valid:
                    local_list.append(sub_list)  
        aux_list.append(local_list)

    ball_arr_coords = itertools.product(*aux_list)
    
    for tup in ball_arr_coords:
        ball_arr = []
        for row in tup:
            ball_row = [0 for j in range(n)]
            for p in row:
                ball_row[p] += 1
            ball_arr.append(ball_row.copy())
        ball_arrangements.append(ball_arr)

    # Create all SpuerMLQs with n columns of given shape,epsilon & compute the polynomial
    poly = 0

    for balls in ball_arrangements:
        SuperM = SupersymmetricMultilineQueue(balls.copy(),epsilon)
        SuperM.pair(0)
        r=0.2
        poly += SuperM.xy_weight()*SuperM.comaj()
#         if SuperM.xy_weight() == exponent(SuperM.xs,[0,1,1,1,2]):
#             SuperM.draw(r,"numbers")

    return(poly)
# Other methods

def exponent(xs,l):
    ans = xs[0]**0
    for i in range(len(l)):
        m = int(l[i])
        ans = ans*xs[i]**m
    return(ans)